In [165]:
import scipy as sp
import numpy as np
import sympy as sy
import scipy.integrate as spi
from sympy.abc import t,x,y, z
import scipy.optimize as spo
from sympy.utilities.lambdify import lambdify 
import itertools as it
import time

In [166]:
#construct homotopy
def H(t, G, F, gamma):
    return [(1 - t)*G[i] + gamma*t*F[i] for i in range(len(G))]

#for degree 3 polynomial, do we need to generalise for more than one variable??
def G(x_array):
    func_listG = [i**3 - 1 for i in x_array]
    return func_listG

#construct inital starting systems for different number of starting systems
def F3(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi*x_array[2]**2, x_array[1]**3 + x_array[2]**2 + 8*x_array[0], x_array[2]**3 \
            + x_array[0]*x_array[1]*x_array[2] + np.pi]

def F2predictortest(x_array):
    return [x_array[0] - 10, x_array[1] - 10]

def F2(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi, x_array[1]**3 + 8*x_array[0] + 76]

def F2Easy(x_array):
    return [x_array[0]**2  + np.pi, x_array[1]**2 - 76]

def F1(x_array):
    return [x_array[0]**3 + np.pi + x_array[0]**2 - x_array[0]]

def gamma_generate():
    real = np.random.rand()
    im = np.sqrt(1- real**2)
    return real + im*1j

def G_roots_find(n):
    root_list = [1, np.exp(1j*2*np.pi/3), np.exp(1j*2*np.pi*2/3)]
    return [i for i in it.product(root_list, repeat = n)]

In [167]:
def Homotopy_Continuation(t, x_array, F, N, tolerance = 1e-10, tolerance_zero = 1e-10, ratio_tolerance = 1e-5,\
                          N_ratio_tolerance = 50, debug = False):
    
    time_start = time.time()
    
    #count the number of roots
    num = 0
    
    delta_t = 1/N
    n = len(x_array)
    gamma = gamma_generate()
    gamma= 0.6 + 0.8j
    G_roots = G_roots_find(n)
    
    #construct homotopy
    H_func = H(t, G(x_array), F(x_array), gamma)
    
    #first derivative of H
    H_diff_x = sy.Matrix([[H_func[i].diff(x_array[j]) for i in range(len(x_array))] for j in range(len(x_array))])

    determinant_H = H_diff_x.det()
    
    #derivative with respect to t
    H_diff_t = sy.Matrix([H_func[i].diff(t) for i in range(len(x_array))])
    
    #check determinant is not zero so can invert
    if abs(determinant_H) == 0:
        raise TypeError('The determinant of H is zero!')
        
    det_H = lambdify((t, x_array), determinant_H)
    
    H_diff_x_inv = H_diff_x**-1
    H_diff_inv = -H_diff_x_inv*H_diff_t
    
    H_Hdiffprime = H_diff_x_inv*sy.Matrix(H_func)
    
    H_Hdiffprime = lambdify((t, x_array), [H_Hdiffprime[i] for i in range(len(H_Hdiffprime))])
        
    H_prime = lambdify((t, x_array), [H_diff_inv[i] for i in range(len(H_diff_inv))])
    
    Hprime_x = lambdify((x_array,t), [H_diff_x[i] for i in range(len(H_diff_x))])    
    H_func_1d = lambdify((x_array,t), H_func)
    

    x_old_arrays = []
    x_olds = []
    
    #run for all roots of starting system
    for x_old in G_roots:
        x_old_array = []
        num += 1
            
        t_n = 0
        
        #run for all steps starting at t=0 ending at t=1
        while t_n < 1:

            x_old_array.append(x_old)
            t_old = t_n
            t_n += delta_t
            
            if n == 1:
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[-1][-1]
                
                H_func_1 = lambdify((x,t), H_func_1d([x], t)[0])
                Hprime_x_1 = lambdify((x,t), Hprime_x([x], t)[0])

                #x_old = [spo.newton(H_func_1, sol_x, fprime = Hprime_x_1, args=(t_n, ))]
                
            else:
                if abs(det_H(t_n, x_old)) < tolerance_zero:
                    raise TypeError('The determinant of H is zero!')
                
                #perform RK4 method
                x_old_predictor=x_old
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[:,-1]
                    
            #newton's method
            x_old = sol_x
            ratio = np.full(n, 1)
            N_ratio = 0
            delta = np.full(n, ratio_tolerance)
            
            #tolerance criteria for step size in Newton's Method
            while max(ratio) > ratio_tolerance and N_ratio < N_ratio_tolerance:
                if abs(det_H(t_n, x_old)) < tolerance_zero:
                    raise TypeError('The determinant of H is zero!')
                x_old_intermediate = x_old - H_Hdiffprime(t_n, x_old)
                
                delta_old = delta
                delta = abs(x_old_intermediate - x_old)
                ratio = [delta[j]/(delta_old[j] + 1e-10) for j in range(n)]
                x_old = x_old_intermediate
                N_ratio += 1
                #x_old = spo.newton(H_func, sol_x, fprime = Hprime_x, args=(t_n, ))
            if debug:

                print('Newtons method took ', N_ratio, ' at step', t_old, " to ", t_n  )
                print('Predictor started at', x_old_predictor)
                print('predicted solution', sol_x)
                print('reached solution', x_old)
        #check root is found
        remainder = list(map(abs, F(x_old))) 

        #if all(rem < tolerance for rem in remainder) is True:
        if True:
            x_old = [x_old[i].real if abs(x_old[i].imag) < tolerance_zero else x_old[i] for i in range(len(x_old))]
            
            print('Root {}, \n{}, \nAccuracy {}!!\n'.format(num, x_old, remainder))
            x_olds.append(x_old)
            x_old_arrays.append(x_old_array)

    time_end = time.time()
    
    print('It took {} s to run!'.format(time_end - time_start))
    
    return x_olds, x_old_arrays

In [174]:
x_sols, x_sols_arrays = Homotopy_Continuation(t, [x,y,z], F3, N=5, tolerance = 1, N_ratio_tolerance= 50, ratio_tolerance= 1e-3,debug=False)

Root 1, 
[(2.3691919954891083+0.9462690838774679j), (1.0504058022080858+2.5125144052837047j), (0.0053823031781055726+0.45129835893952713j)], 
Accuracy [0.02845169036914106, 0.03470603841203209, 0.0058520293282755455]!!

Root 2, 
[(-2.323590975883396+1.009557579528518j), (-1.046066435051285+2.5242266460247085j), (0.021057602010586977-0.45818427464292516j)], 
Accuracy [0.21280095870837423, 0.2652587220274862, 0.04554194359017099]!!

Root 3, 
[(2.3676997959311756-0.945045836275847j), (1.0506186074833652-2.514298792964845j), (0.006659756665282005-0.4516310652479465j)], 
Accuracy [0.02270658226660794, 0.027842259746815846, 0.004686723902000976]!!

Root 4, 
[(-1.9219224401723072+1.4687834435146638j), (2.3906384747392173-0.4010715629882845j), (2.0707383839937252-1.1923833783833937j)], 
Accuracy [3.652994808745098e-13, 1.706135029904003e-13, 1.364590109437335e-13]!!

Root 5, 
[(-2.327286228991342-0.9992897206236809j), (-1.0368581245145767-2.528458853829756j), (0.021646162508663365+0.4506420578

In [11]:
coeff = [5,0,2,np.pi-6]
np.roots(coeff)

array([-0.33583382+0.85927473j, -0.33583382-0.85927473j,
        0.67166764+0.j        ])

In [7]:
coeff = [9,10,0,3]
np.roots(coeff)

array([-1.30641692+0.j        ,  0.0976529 +0.49559532j,
        0.0976529 -0.49559532j])

In [8]:
np.full(1,3)

array([3])

In [34]:
abs(5 + 9j)

10.295630140987

In [ ]:
[1,2]